[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openlayer-ai/openlayer-python/blob/main/examples/tracing/google-adk/google_adk_tracing.ipynb)

# Google ADK Tracing with Openlayer

This notebook demonstrates how to trace Google Agent Development Kit (ADK) agents with Openlayer.

## Features

- **Full Agent Tracing**: Capture agent execution, LLM calls, and tool usage
- **Token Usage Tracking**: Automatically captures prompt, completion, and total tokens
- **All 6 ADK Callbacks**: Trace before_agent, after_agent, before_model, after_model, before_tool, after_tool
- **Google Cloud Coexistence**: Use both Google Cloud telemetry (Cloud Trace) AND Openlayer simultaneously

## Prerequisites

Install the required packages:
```bash
pip install openlayer google-adk wrapt
```


In [ ]:
!pip install google-adk wrapt

## Setup

First, configure your Openlayer credentials and Google Cloud credentials:


In [ ]:
import os

# Openlayer configuration
os.environ["OPENLAYER_API_KEY"] = "your-api-key"
os.environ["OPENLAYER_INFERENCE_PIPELINE_ID"] = "your-pipeline-id"

# Google AI API configuration (Option 1: Using Google AI Studio)
# Get your API key from: https://aistudio.google.com/apikey
os.environ["GOOGLE_API_KEY"] = "your-google-api-key"

# Google Cloud Vertex AI configuration (Option 2: Using Google Cloud)
# Uncomment these if you're using Vertex AI instead of Google AI
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "path/to/your/service-account-key.json"
# os.environ["GOOGLE_CLOUD_PROJECT"] = "your-project-id"
# os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"


## Enable Google ADK Tracing

Enable tracing before creating any agents. This patches Google ADK globally to send traces to Openlayer.

**Note:** By default, ADK's built-in OpenTelemetry tracing remains active, allowing you to send data to both Google Cloud (Cloud Trace, Cloud Monitoring) AND Openlayer. If you only want Openlayer, use `trace_google_adk(disable_adk_otel=True)`.


In [ ]:
from openlayer.lib.integrations import trace_google_adk

# Enable tracing (must be called before creating agents)
trace_google_adk()

## Example 1: Basic Agent with LLM Calls

Create a simple agent that responds to user queries:


In [ ]:

from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

# Setup constants
APP_NAME = "openlayer_demo"
USER_ID = "user_123"
SESSION_ID = "session_123"

# Create session service (shared across examples)
session_service = InMemorySessionService()

# Create a basic agent
agent = LlmAgent(
    model="gemini-2.5-flash",
    name="Assistant",
    instruction="You are a helpful assistant. Provide concise and accurate responses."
)

# Create runner
runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service
)

# Define async function to run the agent
async def run_basic_agent():
    # Create session
    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )
    
    # Run the agent
    query = "What is the capital of France?"
    content = types.Content(role='user', parts=[types.Part(text=query)])
    
    # Process events and get response
    async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
        if event.is_final_response() and event.content:
            final_answer = event.content.parts[0].text.strip()

# Run the async function
await run_basic_agent()

## Example 2: Agent with Tools/Functions

Create an agent with custom tools that can be called during execution:


In [ ]:
import os


# Define custom tools as regular Python functions
def get_weather(city: str) -> str:
    """Retrieves the current weather report for a specified city.
    
    Args:
        city: The name of the city for which to retrieve the weather report.
    
    Returns:
        str: Weather report or error message.
    """
    if city.lower() == "san francisco":
        return "The weather in San Francisco is sunny with a temperature of 72°F (22°C)."
    else:
        return f"Sorry, weather information for '{city}' is not available."

def calculate(expression: str) -> str:
    """Evaluates a mathematical expression.
    
    Args:
        expression: A mathematical expression to evaluate.
    
    Returns:
        str: Calculation result or error message.
    """
    try:
        result = eval(expression)
        return f"The result is {result}"
    except Exception as e:
        return f"Error: {str(e)}"

# Use different session IDs for tool agent
TOOL_USER_ID = "user_456"
TOOL_SESSION_ID = "session_456"

# Create agent with tools (pass functions directly)
tool_agent = LlmAgent(
    model="gemini-2.5-flash",
    name="ToolAgent",
    instruction="You are a helpful assistant with access to weather and calculation tools. Use them when appropriate.",
    tools=[get_weather, calculate]
)

# Create runner for tool agent (reuse the session_service)
tool_runner = Runner(
    agent=tool_agent,
    app_name=APP_NAME,
    session_service=session_service
)

# Define async function to run the tool agent
async def run_tool_agent():
    # Create session
    await session_service.create_session(
        app_name=APP_NAME,
        user_id=TOOL_USER_ID,
        session_id=TOOL_SESSION_ID
    )
    
    # Run the agent with a query that requires tool use
    query = "What's the weather in San Francisco? Also, what is 15 * 24?"
    content = types.Content(role='user', parts=[types.Part(text=query)])
    
    # Process events and get response
    async for event in tool_runner.run_async(
        user_id=TOOL_USER_ID,
        session_id=TOOL_SESSION_ID,
        new_message=content
    ):
        if event.is_final_response() and event.content:
            final_answer = event.content.parts[0].text.strip()

# Run the async function
await run_tool_agent()


## Example 3: Agent with All 6 Callbacks

Google ADK supports 6 types of callbacks that allow you to observe, customize, and control agent behavior. Openlayer automatically traces all of them:

| Callback | Description | When Called |
|----------|-------------|-------------|
| `before_agent_callback` | Agent pre-processing | Before the agent starts its main work |
| `after_agent_callback` | Agent post-processing | After the agent finishes all its steps |
| `before_model_callback` | LLM pre-call | Before sending a request to the LLM |
| `after_model_callback` | LLM post-call | After receiving a response from the LLM |
| `before_tool_callback` | Tool pre-execution | Before executing a tool |
| `after_tool_callback` | Tool post-execution | After a tool finishes |

Reference: https://google.github.io/adk-docs/callbacks/


In [ ]:
from typing import Any, Dict, Optional

from google.adk.tools import ToolContext
from google.adk.models import LlmRequest, LlmResponse
from google.adk.tools.base_tool import BaseTool
from google.adk.agents.callback_context import CallbackContext

# ============================================================================
# Define all 6 callback functions
# ============================================================================

# 1. Before Agent Callback
# Called before the agent starts processing a request
def before_agent_callback(callback_context: CallbackContext, **_kwargs: Any) -> Optional[Any]:
    """
    Called before the agent starts its main work.
    
    Use cases:
    - Input validation
    - Session initialization
    - Logging request start
    - Adding default context
    """
    print(f"[before_agent] Agent '{callback_context.agent_name}' starting")  # noqa: T201
    print(f"[before_agent] Invocation ID: {callback_context.invocation_id}")  # noqa: T201
    # Return None to allow the agent to proceed normally
    # Return a Content object to skip the agent and return that content directly
    return None


# 2. After Agent Callback
# Called after the agent finishes processing
def after_agent_callback(callback_context: CallbackContext, **_kwargs: Any) -> Optional[Any]:
    """
    Called after the agent has finished all its steps.
    
    Use cases:
    - Response post-processing
    - Logging request completion
    - Cleanup operations
    - Analytics
    """
    print(f"[after_agent] Agent '{callback_context.agent_name}' finished")  # noqa: T201
    # Return None to use the agent's original response
    # Return a Content object to replace the agent's response
    return None


# 3. Before Model Callback
# Called before each LLM call
def before_model_callback(
    _callback_context: CallbackContext,
    llm_request: LlmRequest,
    **_kwargs: Any
) -> Optional[LlmResponse]:
    """
    Called before sending a request to the LLM.
    
    Use cases:
    - Request modification (add system prompts)
    - Content filtering / guardrails
    - Caching (return cached response)
    - Rate limiting
    """
    print(f"[before_model] Calling model: {llm_request.model}")  # noqa: T201
    print(f"[before_model] Request has {len(llm_request.contents)} content items")  # noqa: T201
    # Return None to proceed with the LLM call
    # Return an LlmResponse to skip the LLM and use that response instead
    return None


# 4. After Model Callback
# Called after receiving LLM response
def after_model_callback(
    _callback_context: CallbackContext,
    llm_response: LlmResponse,
    **_kwargs: Any
) -> Optional[LlmResponse]:
    """
    Called after receiving a response from the LLM.
    
    Use cases:
    - Response validation
    - Content filtering
    - Response transformation
    - Logging/analytics
    """
    print("[after_model] Received response from LLM")  # noqa: T201
    if hasattr(llm_response, 'usage_metadata') and llm_response.usage_metadata:
        print(f"[after_model] Tokens used: {llm_response.usage_metadata.total_token_count}")  # noqa: T201
    # Return None to use the original response
    # Return a modified LlmResponse to replace it
    return None


# 5. Before Tool Callback
# Called before tool execution
def before_tool_callback(
    tool: BaseTool, 
    _args: Dict[str, Any], 
    _tool_context: ToolContext,
    **_kwargs: Any
) -> Optional[Dict]:
    """
    Called before executing a tool.
    
    Use cases:
    - Argument validation
    - Authorization checks
    - Tool call logging
    - Mocking tool responses for testing
    """
    print(f"[before_tool] Executing tool: {tool.name}")  # noqa: T201
    print(f"[before_tool] Arguments: {args}")  # noqa: T201
    # Return None to proceed with the tool execution
    # Return a dict to skip the tool and use that as the response
    return None


# 6. After Tool Callback
# Called after tool execution
def after_tool_callback(
    tool: BaseTool, 
    _args: Dict[str, Any], 
    _tool_context: ToolContext, 
    tool_response: Dict,
    **_kwargs: Any
) -> Optional[Dict]:
    """
    Called after a tool finishes execution.
    
    Use cases:
    - Response transformation
    - Error handling
    - Logging tool results
    - Caching responses
    """
    print(f"[after_tool] Tool '{tool.name}' completed")  # noqa: T201
    print(f"[after_tool] Response: {tool_response}")  # noqa: T201
    # Return None to use the original tool response
    # Return a modified dict to replace the response
    return None


In [ ]:
# ============================================================================
# Create agent with all callbacks
# ============================================================================

# Define a tool for the callback agent to use
def get_current_time() -> str:
    """Returns the current time.
    
    Returns:
        str: The current time as a formatted string.
    """
    from datetime import datetime
    return f"The current time is {datetime.now().strftime('%H:%M:%S')}"


# Use different session IDs for callback agent
CALLBACK_USER_ID = "user_789"
CALLBACK_SESSION_ID = "session_788"

# Create agent with ALL 6 callbacks
callback_agent = LlmAgent(
    model="gemini-2.5-flash",
    name="CallbackDemoAgent",
    instruction="You are a helpful assistant. Use the get_current_time tool when asked about time.",
    tools=[get_current_time],
    # Register all 6 callbacks
    before_agent_callback=before_agent_callback,
    after_agent_callback=after_agent_callback,
    before_model_callback=before_model_callback,
    after_model_callback=after_model_callback,
    before_tool_callback=before_tool_callback,
    after_tool_callback=after_tool_callback,
)

# Create runner for callback agent
callback_runner = Runner(
    agent=callback_agent,
    app_name=APP_NAME,
    session_service=session_service
)

# Define async function to run the callback agent
async def run_callback_agent():
    # Create session
    await session_service.create_session(
        app_name=APP_NAME,
        user_id=CALLBACK_USER_ID,
        session_id=CALLBACK_SESSION_ID
    )
    
    # Run the agent with a query that will trigger tool use
    query = "What time is it right now?"
    content = types.Content(role='user', parts=[types.Part(text=query)])
    
    # Process events and get response
    async for event in callback_runner.run_async(
        user_id=CALLBACK_USER_ID,
        session_id=CALLBACK_SESSION_ID,
        new_message=content
    ):
        if event.is_final_response() and event.content:
            print(f"Final Response: {event.content.parts[0].text.strip()}")  # noqa: T201

# Run the async function
await run_callback_agent()


### What You'll See in Openlayer

After running the callback agent, you'll see in your Openlayer dashboard:

1. **Agent Step** (`CallbackDemoAgent`):
   - Shows which callbacks are registered
   - Contains all nested steps in chronological order

2. **All Callbacks as Siblings** (direct children of Agent):
   - `Callback: before_agent` - First, before any processing
   - `Callback: before_model` - Before each LLM call
   - `LLM Call: gemini-2.0-flash-exp` - The actual LLM invocation
   - `Callback: after_model` - After each LLM call (includes token counts)
   - `Callback: before_tool` - Before tool execution
   - `Tool: get_current_time` - The actual tool execution
   - `Callback: after_tool` - After tool completion
   - `Callback: after_agent` - Last, after all processing complete

3. **Token Usage** (captured on LLM Call steps):
   - Prompt tokens
   - Completion tokens  
   - Total tokens


## View Traces in Openlayer

After running these examples, you can view the traces in your Openlayer dashboard:

1. Go to https://app.openlayer.com
2. Navigate to your inference pipeline
3. View the traces tab to see:
   - **Agent execution steps** with nested hierarchy
   - **LLM calls** with token counts (prompt, completion, total)
   - **Tool executions** with inputs and outputs
   - **All 6 callback types** traced as separate steps
   - **Latency** for each operation
   - **Complete execution hierarchy** showing the flow

The traces will show the hierarchy of operations in chronological order:
```
Agent: CallbackDemoAgent
├── Callback: before_agent (CallbackDemoAgent)
├── Callback: before_model (CallbackDemoAgent)
├── LLM Call: gemini-2.0-flash-exp
├── Callback: after_model (CallbackDemoAgent)
├── Callback: before_tool (CallbackDemoAgent)
├── Tool: get_current_time
├── Callback: after_tool (CallbackDemoAgent)
├── Callback: before_model (CallbackDemoAgent)
├── LLM Call: gemini-2.0-flash-exp
├── Callback: after_model (CallbackDemoAgent)
└── Callback: after_agent (CallbackDemoAgent)
```

**Note:** All callbacks are direct children of the Agent step, appearing in chronological order alongside LLM calls and tool executions. This provides a clear timeline view of the agent execution flow.


## Disable Tracing

When you're done, you can disable tracing to restore ADK's default behavior:


In [ ]:
from openlayer.lib.integrations import unpatch_google_adk

# Disable tracing
unpatch_google_adk()